In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from itertools import product
from sklearn.metrics import roc_auc_score, roc_curve,  classification_report, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [ ]:
def focal_loss_obj(alpha=0.25, gamma=2.0):
    def fl_obj(preds, dtrain):
        labels = dtrain.get_label()
        preds = 1.0 / (1.0 + np.exp(-preds))
        preds = np.clip(preds, 1e-9, 1 - 1e-9)
        grad = (
            alpha * labels * (1 - preds) ** gamma * (gamma * preds * np.log(preds) + preds - 1) +
            (1 - alpha) * (1 - labels) * preds ** gamma * (gamma * (1 - preds) * np.log(1 - preds) - preds)
        )
        hess = (
            alpha * labels * (1 - preds) ** gamma * preds * (1 - preds) +
            (1 - alpha) * (1 - labels) * preds ** gamma * preds * (1 - preds)
        )
        return grad, hess
    return fl_obj


In [ ]:
# ### Load Dataset ###
Df = pd.read_excel("Input the path of Internal Dataset")

X1=Df.iloc[:,0:-1]
Y=Df.iloc[:,-1]


In [ ]:
### Hyperparameters Grid #####
param_grid = {
    "max_depth": [10, 20, 30, 40, 50],
    "eta": [0.001, 0.01, 0.05, 0.1],
    "subsample": [0.8, 0.9],
    "colsample_bytree": [0.8, 0.9],
    "alpha": [0.25, 0.5],
    "gamma": [1.0, 2.0]
}
grid = list(ParameterGrid(param_grid))

In [ ]:
X = np.array(X1)
y = np.array(Y)

## roc-auc score
ROC_AUC = []
## sensitivity score
SEN = []
## specificity score
SPE = [] 
## Weighted F1-score
F1 = [] 
## Weighted Precision score
Pr=[]
## Weighted Recall score
Rew=[]

outer_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_auc_scores = []

for fold_num, (train_idx, test_idx) in enumerate(outer_skf.split(X, y), 1):
    print(f"Outer Fold {fold_num}")
    
    X_train_outer1, X_test1 = X[train_idx], X[test_idx]
    y_train_outer, y_test = y[train_idx], y[test_idx]

    ### Nornalization ###
    scaler=StandardScaler().fit(X_train_outer1)
    X_train_outer=scaler.transform(X_train_outer1)
    X_test=scaler.transform(X_test1)

    #######################################################################
    ### Inner CV (75% of data) for hyperparameter tuning ###
    inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_inner_auc = 0
    best_inner_params = None

    for params_candidate in grid:
        inner_fold_aucs = []

        for inner_train_idx, inner_val_idx in inner_skf.split(X_train_outer, y_train_outer):
            X_train_inner, X_val_inner = X_train_outer[inner_train_idx], X_train_outer[inner_val_idx]
            y_train_inner, y_val_inner = y_train_outer[inner_train_idx], y_train_outer[inner_val_idx]

            ### Class Weight ###
            pos = np.sum(y_train_inner == 1)
            neg = np.sum(y_train_inner == 0)
            scale_pos_weight = neg / pos

            dtrain_inner = xgb.DMatrix(X_train_inner, label=y_train_inner)
            dval_inner = xgb.DMatrix(X_val_inner, label=y_val_inner)

            model_inner = xgb.train(
                params={
                    "max_depth": params_candidate["max_depth"],
                    "eta": params_candidate["eta"],
                    "subsample": params_candidate["subsample"],
                    "colsample_bytree": params_candidate["colsample_bytree"],
                    "eval_metric": "auc",
                    "scale_pos_weight": scale_pos_weight
                },
                dtrain=dtrain_inner,
                num_boost_round=1000,
                obj=focal_loss_obj(alpha=params_candidate["alpha"], gamma=params_candidate["gamma"]),
                evals=[(dval_inner, "validation")],
                early_stopping_rounds=100,
                verbose_eval=False
            )

            y_val_pred = model_inner.predict(dval_inner)
            auc = roc_auc_score(y_val_inner, y_val_pred)
            inner_fold_aucs.append(auc)

        mean_inner_auc = np.mean(inner_fold_aucs)

        if mean_inner_auc > best_inner_auc:
            best_inner_auc = mean_inner_auc
            best_inner_params = params_candidate
    #######################################################################

    #######################################################################
    ### Model performance with the tunned hyperparameters ###
    ## Class weight ##
    pos = np.sum(y_train_outer == 1)
    neg = np.sum(y_train_outer == 0)
    scale_pos_weight = neg / pos

    dtrain_outer = xgb.DMatrix(X_train_outer, label=y_train_outer)
    dtest_outer = xgb.DMatrix(X_test, label=y_test)

    final_model = xgb.train(
        params={
            "max_depth": best_inner_params["max_depth"],
            "eta": best_inner_params["eta"],
            "subsample": best_inner_params["subsample"],
            "colsample_bytree": best_inner_params["colsample_bytree"],
            "eval_metric": "auc",
            "scale_pos_weight": scale_pos_weight
        },
        dtrain=dtrain_outer,
        num_boost_round=1000,
        obj=focal_loss_obj(alpha=best_inner_params["alpha"], gamma=best_inner_params["gamma"]),
        evals=[(dtest_outer, "test")],
        early_stopping_rounds=100,
        verbose_eval=False
    )

    y_test_pred = final_model.predict(dtest_outer)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred_optimal = (y_test_pred >= optimal_threshold).astype(int)
    
    ## Performance metrics evaluation 
    report = classification_report(y_test, y_pred_optimal, output_dict=True)
    print(classification_report(y_test, y_pred_optimal))
    auc = roc_auc_score(y_test, y_pred_optimal)
    Sensitivity=report['1']['recall']
    Specificity=report['0']['recall']
    F1score=report['weighted avg']['f1-score']
    Recall_w=report['weighted avg']['recall']
    Precision=report['weighted avg']['precision']
  
    ROC_AUC.append(auc)
    SEN.append(Sensitivity)
    SPE.append(Specificity)
    F1.append(F1score)
    Pr.append(Precision)
    Rew.append(Recall_w) 


print(np.mean(ROC_AUC), np.mean(SEN), np.mean(SPE), np.mean(F1), np.mean(Pr), np.mean(Rew))
print(np.std(ROC_AUC), np.std(SEN), np.std(SPE), np.std(F1), np.std(Pr), np.std(Rew))

In [ ]:
confidence_interval_ROC_AUC = np.percentile(ROC_AUC, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_ROC_AUC}")

confidence_interval_SEN = np.percentile(SEN, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_SEN}")

confidence_interval_SPE = np.percentile(SPE, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_SPE}")

confidence_interval_F1 = np.percentile(F1, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_F1}")

confidence_interval_Pr = np.percentile(Pr, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_Pr}")

confidence_interval_Rew = np.percentile(Rew, [2.5, 97.5])
print(f"Bootstrap 95% Confidence Interval: {confidence_interval_Rew}")